**Задание 1. Написать теггер на данных с русским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы, сделать выводы


In [1]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu 
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-12-29 16:25:14--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   187MB/s    in 0.2s    

2022-12-29 16:25:15 (187 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [40736581/40736581]

--2022-12-29 16:25:15--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108

In [2]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

import warnings
warnings.filterwarnings("ignore")

In [4]:
data_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [5]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [6]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [7]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



Лучший результат у комбинации "Bigram and Unigram Tagger"

In [12]:
from nltk.tag import SequentialBackoffTagger

In [13]:
class NamesTagger(SequentialBackoffTagger):
    def __init__(self, model, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.model = model
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        return self.model.tag([word])[0][1]
            
nt = NamesTagger(trigram_tagger)
print(nt.tag(['составляло', 'период'])) 
print(nt.tag(['Армении'])) 
print(nt.tag(['.']))

[('составляло', 'VERB'), ('период', 'NOUN')]
[('Армении', 'PROPN')]
[('.', 'PUNCT')]


In [14]:
nt.accuracy(fdata_test)

0.822696790155609

**Задание 2. Проверить, насколько хорошо работает NER**

Данные брать из http://www.labinform.ru/pub/named_entities/
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы.
* передаём в сетку токен и его соседей.
* передаём в сетку только токен.
* свой вариант.
3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [35]:
!/usr/local/bin/spacy download ru_core_news_sm

2022-12-29 17:38:40.344330: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 2.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [28]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-12-29 17:37:26--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.50MB/s    in 1.2s    

2022-12-29 17:37:28 (1.50 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [ ]:
!unzip collection5.zip

In [25]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.9 MB/s 


In [26]:
import corus
from corus import load_ne5

nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [30]:
records = load_ne5('Collection5/')
document = next(records).text
document

'Из-за подчиненного-взяточника должностей лишились трое полицейских начальников (Воронеж)\r\n\r\nВ Воронеже участковый уволен за получение взятки, должностей лишились также трое его начальников, сообщили корреспонденту ИА REGNUM в пресс-службе ГУ МВД РФ по региону.\r\n\r\nСотрудники оперативно-разыскной части собственной безопасности ГУ МВД России по Воронежской области пресекли факт получения незаконного денежного вознаграждения полицейским ОП № 5 УМВД России по г. Воронежу. Старший участковый уполномоченный требовал от жителя областного центра 30 тысяч рублей за не привлечение ее к уголовной ответственности. По материалам проверки, проведенной сотрудниками ОРЧ СБ ГУ МВД России по Воронежской области принято процессуальное решение.\r\n\r\nВ ГУ МВД России по Воронежской области проведена служебная проверка. Полицейский уволен из органов внутренних дел.\r\n\r\nКроме того, за ненадлежащий контроль за подчиненным из органов внутренних дел уволены заместитель начальника ОП № 5 УМВД России 

In [31]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Воронеж', 'ORGANIZATION'),
 ('Воронеже', 'PERSON'),
 ('ГУ', 'ORGANIZATION'),
 ('Кроме', 'PERSON'),
 ('МВД России', 'ORGANIZATION'),
 ('России', 'PERSON'),
 ('Самарской', 'PERSON'),
 ('Татарстане', 'ORGANIZATION'),
 ('УМВД России', 'ORGANIZATION'),
 ('Ульяновске', 'ORGANIZATION')}

In [32]:
import spacy
from spacy import displacy

In [36]:
nlp = spacy.load('ru_core_news_sm')

In [37]:
ny_bb = document
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [39]:
for token in article:
    print(token.text, token.pos_, token.dep_)

Из ADP case
- ADP case
за ADP case
подчиненного NOUN obl
- NOUN obl
взяточника NOUN obl
должностей NOUN obj
лишились VERB ROOT
трое NUM nummod:gov
полицейских ADJ amod
начальников NOUN nsubj
( PUNCT punct
Воронеж PROPN parataxis
) PUNCT punct


 SPACE dep
В ADP case
Воронеже PROPN obl
участковый NOUN nsubj:pass
уволен VERB conj
за ADP case
получение NOUN obl
взятки NOUN nmod
, PUNCT punct
должностей NOUN obj
лишились VERB conj
также ADV advmod
трое NUM nummod:gov
его PRON nmod
начальников NOUN nsubj
, PUNCT punct
сообщили VERB parataxis
корреспонденту NOUN iobj
ИА PROPN nmod
REGNUM PROPN appos
в ADP case
пресс NOUN obl
- NOUN obl
службе NOUN obl
ГУ PROPN nmod
МВД PROPN nmod
РФ PROPN nmod
по ADP case
региону NOUN nmod
. PUNCT punct


 SPACE dep
Сотрудники NOUN nsubj
оперативно ADJ amod
- ADJ amod
разыскной ADJ amod
части NOUN nmod
собственной ADJ amod
безопасности NOUN nmod
ГУ PROPN nmod
МВД PROPN nmod
России PROPN nmod
по ADP case
Воронежской ADJ amod
области NOUN nmod
пресекли VERB RO

In [40]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from razdel import tokenize

In [42]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [43]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [44]:
df_words['tag'].value_counts()

OUT         218911
PER          21200
ORG          13623
LOC           4551
GEOPOLIT      4347
MEDIA         2480
Name: tag, dtype: int64

In [45]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input, Bidirectional,Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [46]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [47]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(2048)
valid_data = valid_data.batch(2048)

In [48]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [49]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len
    )

text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [50]:
t=np.unique(encoder.inverse_transform(valid_y),return_counts=True)[1]
t=t/t.sum()

In [51]:
model = tf.keras.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(300, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
  ])

In [52]:
model.compile(optimizer='adam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
98/98 [==============================] - 11s 87ms/step - loss: 0.7604 - accuracy: 0.8182 - val_loss: 0.4190 - val_accuracy: 0.8605
Epoch 2/5
98/98 [==============================] - 6s 60ms/step - loss: 0.2743 - accuracy: 0.9079 - val_loss: 0.2938 - val_accuracy: 0.9285
Epoch 3/5
98/98 [==============================] - 8s 81ms/step - loss: 0.1557 - accuracy: 0.9535 - val_loss: 0.2732 - val_accuracy: 0.9392
Epoch 4/5
98/98 [==============================] - 5s 50ms/step - loss: 0.1240 - accuracy: 0.9623 - val_loss: 0.2687 - val_accuracy: 0.9403
Epoch 5/5
98/98 [==============================] - 5s 50ms/step - loss: 0.1153 - accuracy: 0.9634 - val_loss: 0.2665 - val_accuracy: 0.9411


In [53]:
pred=model.predict(valid_data)

33/33 [==============================] - 0s 10ms/step


In [54]:
list(zip(encoder.classes_,pred.mean(axis=0)/t))

[('GEOPOLIT', 0.9660583205350283),
 ('LOC', 0.9140227697502329),
 ('MEDIA', 0.9152980584028253),
 ('ORG', 0.8899091011574607),
 ('OUT', 1.030093689454892),
 ('PER', 0.7989871401380951)]

In [55]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6,activation='softmax')
])

In [56]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [57]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
98/98 [==============================] - 45s 367ms/step - loss: 0.8336 - accuracy: 0.8198 - val_loss: 0.4078 - val_accuracy: 0.8602
Epoch 2/5
98/98 [==============================] - 34s 351ms/step - loss: 0.3022 - accuracy: 0.9023 - val_loss: 0.2745 - val_accuracy: 0.9173
Epoch 3/5
98/98 [==============================] - 36s 369ms/step - loss: 0.1954 - accuracy: 0.9376 - val_loss: 0.2445 - val_accuracy: 0.9327
Epoch 4/5
98/98 [==============================] - 34s 350ms/step - loss: 0.1582 - accuracy: 0.9521 - val_loss: 0.2334 - val_accuracy: 0.9386
Epoch 5/5
98/98 [==============================] - 37s 380ms/step - loss: 0.1413 - accuracy: 0.9583 - val_loss: 0.2283 - val_accuracy: 0.9399
